## Pipeline Parameters

In [0]:
dbutils.widgets.text("target_table_name", 'kdayno_gold_SP500_stock_prices_avg_agg')

# Audit Parameters
dbutils.widgets.text("job_id", "")
dbutils.widgets.text("job_name", "")
dbutils.widgets.text("job_start_date", "")
dbutils.widgets.text("job_start_datetime", "")
dbutils.widgets.text("task_run_id", "")
dbutils.widgets.text("task_name", "")

In [0]:
%run ../utils/audit_logger

In [0]:
# Standard library imports
import os

# Third-party library imports
from dotenv import load_dotenv

In [0]:
load_dotenv()

catalog_name = os.getenv('DATABRICKS_CATALOG_NAME')
schema_name = os.getenv('DATABRICKS_SCHEMA_NAME')

source_table_name = 'kdayno_silver_SP500_stock_prices'
target_table_name = dbutils.widgets.get("target_table_name")

# Audit Variables
job_id = dbutils.widgets.get('job_id')
job_name = dbutils.widgets.get('job_name')
job_start_date = dbutils.widgets.get('job_start_date')
job_start_datetime = dbutils.widgets.get('job_start_datetime')
task_run_id = dbutils.widgets.get('task_run_id')
task_name = dbutils.widgets.get('task_name')

## Pipeline Audit

In [0]:
audit_logger(job_id, job_name, 'None', job_start_date, job_start_datetime, task_run_id,  task_name, source_table_name, target_table_name)

## ETL

In [0]:
print(f"Running aggregate process... ")

SP500_stock_prices_avg_agg_df = spark.sql(
    f"""
    SELECT 
        ticker_symbol
        , company_name
        , trading_date
        , ROUND(AVG((open_price + close_price + highest_price + lowest_price)) / 4, 2) AS average_trading_price
    FROM {catalog_name}.{schema_name}.{source_table_name}
    GROUP BY ticker_symbol, company_name, trading_date
    """)

In [0]:
print(f"Loading data to target table: {catalog_name}.{schema_name}.{target_table_name}")

(SP500_stock_prices_avg_agg_df.write.format('delta')
                                .mode('overwrite')
                                .saveAsTable(f'{catalog_name}.{schema_name}.{target_table_name}'))